In [ ]:
import numpy as np
import zephyr.backend
import zephyr.middleware
import SimPEG

In [ ]:
nx = 100
nz = 200
cBackground = 2500.
cAnomaly = -500.

cTrue = cBackground * np.ones((nz,nx))
cTrue[(nz/2)-10:(nz/2)+10,(nx/2)-10:(nx/2)+10] += cAnomaly

sz = np.arange(25, nz-24, 1)
sx = 25. * np.ones((len(sz),))
rz = np.arange(25, nz-24, 1)
rx = (nx - 25.) * np.ones((len(rz),))

geom = {
    'src':      np.vstack([sx, sz]).T,
    'rec':      np.vstack([rx, rz]).T,
}

systemConfig = {
    'dx':       1.,                         # m
    'dz':       1.,                         # m
    'c':        cTrue,                      # m/s
    'rho':      1.,                         # kg/m^3
    'nx':       nx,                         # count
    'nz':       nz,                         # count
    'freqs':    np.arange(50, 450, 50),     # Hz
    'disc':     zephyr.backend.MiniZephyr,  # discretization
    'geom':     geom,                       # dictionary
}

In [ ]:
problem = zephyr.middleware.HelmBaseProblem(systemConfig)
survey  = zephyr.middleware.HelmBaseSurvey(systemConfig)
problem.pair(survey)

In [ ]:
dObs = survey.dpred()
survey.dobs = dObs
survey.std = 1.

dmisfit = SimPEG.DataMisfit.l2_DataMisfit(survey)
dmisfit.Wd = 1.

opt = SimPEG.Optimization.SteepestDescent(maxIter = 5)
reg = SimPEG.Regularization.Tikhonov(problem.mesh)

invProb = SimPEG.InvProblem.BaseInvProblem(dmisfit, reg, opt)
inv = SimPEG.Inversion.BaseInversion(invProb)

In [ ]:
mOpt = inv.run(cBackground)